In [1]:
import pandas as pd
import os
import numpy as np
import plotly.express as px

In [10]:
accident_df=pd.read_csv("Resources/accident-data.csv")
accident_df.head()

,accident_index,accident_year,accident_reference,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,...,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area
0,2020010219808,2020,10219808,-0.254001,51.462262,3,1,1,04/02/2020,3,...,6,0,9,9,1,9,9,0,0,1
1,2020010220496,2020,10220496,-0.139253,51.470327,3,1,2,27/04/2020,2,...,6,0,0,4,1,1,1,0,0,1
2,2020010228005,2020,10228005,-0.178719,51.529614,3,1,1,01/01/2020,4,...,6,0,0,0,4,1,2,0,0,1
3,2020010228006,2020,10228006,-0.001683,51.541210,2,1,1,01/01/2020,4,...,6,0,0,4,4,1,1,0,0,1
4,2020010228011,2020,10228011,-0.137592,51.515704,3,1,2,01/01/2020,4,...,5,0,0,0,4,1,1,0,0,1


In [38]:
accident_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91199 entries, 0 to 91198
Data columns (total 29 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   accident_index                           91199 non-null  object 
 1   accident_year                            91199 non-null  int64  
 2   accident_reference                       91199 non-null  object 
 3   longitude                                91185 non-null  float64
 4   latitude                                 91185 non-null  float64
 5   accident_severity                        91199 non-null  int64  
 6   number_of_vehicles                       91199 non-null  int64  
 7   number_of_casualties                     91199 non-null  int64  
 8   date                                     91199 non-null  object 
 9   day_of_week                              91199 non-null  int64  
 10  time                                     91199

In [41]:
cleaned = accident_df.drop(columns=["accident_year","accident_reference","first_road_class","first_road_number",\
                                   "second_road_class","second_road_number"])

In [42]:
cleaned

,accident_index,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,road_type,...,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,severity,road_surface
0,2020010219808,-0.254001,51.462262,3,1,1,04/02/2020,3,09:00,6,...,9,9,1,9,9,0,0,1,Slight,unknown (self reported)
1,2020010220496,-0.139253,51.470327,3,1,2,27/04/2020,2,13:55,6,...,0,4,1,1,1,0,0,1,Slight,Dry
2,2020010228005,-0.178719,51.529614,3,1,1,01/01/2020,4,01:25,6,...,0,0,4,1,2,0,0,1,Slight,Wet or damp
3,2020010228006,-0.001683,51.541210,2,1,1,01/01/2020,4,01:50,6,...,0,4,4,1,1,0,0,1,Serious,Dry
4,2020010228011,-0.137592,51.515704,3,1,2,01/01/2020,4,02:25,6,...,0,0,4,1,1,0,0,1,Slight,Dry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91194,2020991027064,-2.926320,56.473539,2,2,1,12/08/2020,4,14:30,6,...,0,0,1,1,1,0,0,1,Serious,Dry
91195,2020991029573,-4.267565,55.802353,3,1,1,13/11/2020,6,15:05,9,...,0,0,1,1,1,0,0,1,Slight,Dry
91196,2020991030297,-2.271903,57.186317,2,2,1,15/04/2020,4,12:42,6,...,0,0,1,1,1,0,0,2,Serious,Dry
91197,2020991030900,-3.968753,55.950940,3,2,1,15/12/2020,3,14:00,6,...,0,0,1,1,1,0,0,1,Slight,Dry


In [43]:
cleaned.loc[cleaned["accident_severity"] ==3,"severity"] = "Slight"
cleaned.loc[cleaned["accident_severity"] ==2,"severity"] = "Serious"
cleaned.loc[cleaned["accident_severity"] ==1,"severity"] = "Fatal"

In [44]:
cleaned.loc[cleaned["road_surface_conditions"] ==1,"road_surface"] = "Dry"
cleaned.loc[cleaned["road_surface_conditions"] ==2,"road_surface"] = "Wet or damp"
cleaned.loc[cleaned["road_surface_conditions"] ==3,"road_surface"] = "Snow"
cleaned.loc[cleaned["road_surface_conditions"] ==4,"road_surface"] = "Frost or ice"
cleaned.loc[cleaned["road_surface_conditions"] ==5,"road_surface"] = "Flood over 3cm. deep"
cleaned.loc[cleaned["road_surface_conditions"] ==6,"road_surface"] = "Oil or diesel"
cleaned.loc[cleaned["road_surface_conditions"] ==7,"road_surface"] = "Mud"
cleaned.loc[cleaned["road_surface_conditions"] ==-1,"road_surface"] = "Data missing or out of range"
cleaned.loc[cleaned["road_surface_conditions"] ==9,"road_surface"] = "unknown (self reported)"

1. What do we do with unknown data?
2. Do we want to include actual description instead of number?
3. What columns to drop?
4. How to categorize time? (morning, night, 8:00am, 9:00am)
5. making regression using only categorical data
6. What are our hypotheses?

In [45]:
cleaned

,accident_index,longitude,latitude,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,road_type,...,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,severity,road_surface
0,2020010219808,-0.254001,51.462262,3,1,1,04/02/2020,3,09:00,6,...,9,9,1,9,9,0,0,1,Slight,unknown (self reported)
1,2020010220496,-0.139253,51.470327,3,1,2,27/04/2020,2,13:55,6,...,0,4,1,1,1,0,0,1,Slight,Dry
2,2020010228005,-0.178719,51.529614,3,1,1,01/01/2020,4,01:25,6,...,0,0,4,1,2,0,0,1,Slight,Wet or damp
3,2020010228006,-0.001683,51.541210,2,1,1,01/01/2020,4,01:50,6,...,0,4,4,1,1,0,0,1,Serious,Dry
4,2020010228011,-0.137592,51.515704,3,1,2,01/01/2020,4,02:25,6,...,0,0,4,1,1,0,0,1,Slight,Dry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91194,2020991027064,-2.926320,56.473539,2,2,1,12/08/2020,4,14:30,6,...,0,0,1,1,1,0,0,1,Serious,Dry
91195,2020991029573,-4.267565,55.802353,3,1,1,13/11/2020,6,15:05,9,...,0,0,1,1,1,0,0,1,Slight,Dry
91196,2020991030297,-2.271903,57.186317,2,2,1,15/04/2020,4,12:42,6,...,0,0,1,1,1,0,0,2,Serious,Dry
91197,2020991030900,-3.968753,55.950940,3,2,1,15/12/2020,3,14:00,6,...,0,0,1,1,1,0,0,1,Slight,Dry
